In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objs as go 
import plotly.offline as py 
import math
from collections import Counter

!pip install -U ipython lightgbm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone 

import xgboost as xgb


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install eli5
import eli5
from eli5.sklearn import PermutationImportance


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
print("\nTransaction features: \n")
df_txs_features = pd.read_csv("D:/Financial Data Analytics/Final Project/Dataset/txs_features - Copy.csv")
df_txs_features

print("\nTransaction classes: \n")
df_txs_classes = pd.read_csv("D:/Financial Data Analytics/Final Project/Dataset/txs_classes.csv")
df_txs_classes

print("\nTransaction-Transaction edgelist: \n")
df_txs_edgelist = pd.read_csv("D:/Financial Data Analytics/Final Project/Dataset/txs_edgelist.csv")
df_txs_edgelist


Transaction features: 



,txId,Time step,class,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,...,in_BTC_min,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total
0,3321,1,3,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.534072,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972
1,11108,1,3,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,5.611878,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778
2,51816,1,3,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.456608,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508
3,68869,1,2,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,...,0.308900,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800
4,89273,1,2,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,...,852.164680,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203764,158304003,49,3,-0.165622,-0.139563,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203765,158303998,49,3,-0.167040,-0.139563,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203766,158303966,49,3,-0.167040,-0.139563,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203767,161526077,49,3,-0.172212,-0.139573,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Transaction classes: 



,txId,class
0,3321,3
1,11108,3
2,51816,3
3,68869,2
4,89273,2
...,...,...
203764,158304003,3
203765,158303998,3
203766,158303966,3
203767,161526077,3



Transaction-Transaction edgelist: 



,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206
...,...,...
234350,158365409,157930723
234351,188708874,188708879
234352,157659064,157659046
234353,87414554,106877725


In [4]:
print("\ntxs_features.csv for txId = 272145560\n")
df_txs_features[df_txs_features['txId']==272145560]

print("\ntxs_classes.csv for txId = 272145560\n")
df_txs_classes[df_txs_classes['txId']==272145560]

print("\ntxs_edgelist.csv for txId = 272145560\n")
df_txs_edgelist[(df_txs_edgelist['txId1']==272145560) | (df_txs_edgelist['txId2']==272145560)]


txs_features.csv for txId = 272145560



,txId,Time step,class,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,...,in_BTC_min,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total
105573,272145560,24,1,-0.155493,-0.107012,-1.201369,-0.12197,-0.043875,-0.113002,-0.061584,...,2.7732,2.7732,2.7732,2.7732,2.7732,0.001917,2.770883,1.3864,1.3864,2.7728



txs_classes.csv for txId = 272145560



,txId,class
105573,272145560,1



txs_edgelist.csv for txId = 272145560



,txId1,txId2
123072,272145560,296926618
123272,272145560,272145556
125873,299475624,272145560


In [5]:
list(df_txs_features.columns)

['txId',
 'Time step',
 'class',
 'Local_feature_1',
 'Local_feature_2',
 'Local_feature_3',
 'Local_feature_4',
 'Local_feature_5',
 'Local_feature_6',
 'Local_feature_7',
 'Local_feature_8',
 'Local_feature_9',
 'Local_feature_10',
 'Local_feature_11',
 'Local_feature_12',
 'Local_feature_13',
 'Local_feature_14',
 'Local_feature_15',
 'Local_feature_16',
 'Local_feature_17',
 'Local_feature_18',
 'Local_feature_19',
 'Local_feature_20',
 'Local_feature_21',
 'Local_feature_22',
 'Local_feature_23',
 'Local_feature_24',
 'Local_feature_25',
 'Local_feature_26',
 'Local_feature_27',
 'Local_feature_28',
 'Local_feature_29',
 'Local_feature_30',
 'Local_feature_31',
 'Local_feature_32',
 'Local_feature_33',
 'Local_feature_34',
 'Local_feature_35',
 'Local_feature_36',
 'Local_feature_37',
 'Local_feature_38',
 'Local_feature_39',
 'Local_feature_40',
 'Local_feature_41',
 'Local_feature_42',
 'Local_feature_43',
 'Local_feature_44',
 'Local_feature_45',
 'Local_feature_46',
 'Local_fe

In [6]:
df_txs_features = df_txs_features.dropna()
df_txs_features

,txId,Time step,class,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,...,in_BTC_min,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total
0,3321,1,3,-0.169615,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.534072,0.534072,0.534072,0.534072,0.534072,1.668990e-01,0.367074,0.266986,0.266986,0.533972
1,11108,1,3,-0.137586,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,5.611878,5.611878,5.611878,5.611878,5.611878,5.861940e-01,5.025584,2.805889,2.805889,5.611778
2,51816,1,3,-0.170103,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.456608,0.456608,0.456608,0.456608,0.456608,2.279902e-01,0.228518,0.228254,0.228254,0.456508
3,68869,1,2,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,...,0.308900,8.000000,3.102967,1.000000,9.308900,1.229000e+00,8.079800,4.654400,4.654400,9.308800
4,89273,1,2,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,...,852.164680,852.164680,852.164680,852.164680,852.164680,1.300000e-07,41.264036,0.065016,0.000441,852.164680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202799,194747812,49,3,0.558398,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,...,115.952889,115.952889,115.952889,115.952889,115.952889,1.653300e+00,114.299544,57.976422,57.976422,115.952844
202800,194747925,49,3,0.547658,-0.198956,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,...,114.250098,114.250098,114.250098,114.250098,114.250098,2.035300e-02,114.229700,57.125027,57.125027,114.250053
202801,194748063,49,3,0.543600,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,...,113.606771,113.606771,113.606771,113.606771,113.606771,9.257490e-01,112.680977,56.803363,56.803363,113.606726
202802,194748070,49,3,0.537760,-0.198853,-0.091383,-0.121970,-0.043875,-0.113002,-0.061584,...,112.680977,112.680977,112.680977,112.680977,112.680977,3.026970e-01,112.378235,56.340466,56.340466,112.680932


In [7]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Only scale feature columns, not the label
for column in df_txs_features.columns[168:]:
    if column == 'Class':
        continue  # Don't scale the class label

    feature = np.array(df_txs_features[column]).reshape(-1, 1)
    scaler = MinMaxScaler()
    scaler.fit(feature)
    feature_scaled = scaler.transform(feature)
    df_txs_features[column] = feature_scaled.reshape(1, -1)[0]


MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

...

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

MinMaxScaler()

In [8]:
# remove 'unknown' transactions
data = df_txs_features.loc[(df_txs_features['class'] != 3), 'txId']
df_txs_features_selected = df_txs_features.loc[df_txs_features['txId'].isin(data)]
df_txs_features_selected

,txId,Time step,class,Local_feature_1,Local_feature_2,Local_feature_3,Local_feature_4,Local_feature_5,Local_feature_6,Local_feature_7,...,in_BTC_min,in_BTC_max,in_BTC_mean,in_BTC_median,in_BTC_total,out_BTC_min,out_BTC_max,out_BTC_mean,out_BTC_median,out_BTC_total
3,68869,1,2,-0.114267,-0.184668,-1.201369,0.028105,-0.043875,-0.113002,0.547008,...,2.711586e-05,7.022548e-04,2.723834e-04,8.778200e-05,8.171503e-04,6.113009e-04,7.142783e-04,0.001552,1.552291e-03,8.171446e-04
4,89273,1,2,5.202107,-0.210553,-1.756361,-0.121970,260.090707,-0.113002,-0.061584,...,7.480472e-02,7.480472e-02,7.480472e-02,7.480472e-02,7.480472e-02,6.466160e-11,3.647866e-03,0.000022,1.451405e-07,7.480473e-02
11,293323,1,2,-0.172726,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,3.579195e-06,3.577994e-06,3.578002e-06,3.579195e-06,3.575571e-06,4.715323e-07,3.511341e-06,0.000007,6.780735e-06,3.569892e-06
22,1494462,1,2,-0.172921,-0.158783,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,8.778192e-07,8.766174e-07,8.766262e-07,8.778192e-07,8.741947e-07,1.442451e-06,6.094506e-07,0.000002,1.632382e-06,8.597370e-07
25,1582950,1,2,-0.169967,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,4.198409e-05,4.198289e-05,4.198290e-05,4.198409e-05,4.198047e-05,2.302948e-05,3.817869e-05,0.000080,7.973672e-05,4.197479e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202762,194334585,49,2,-0.039416,-0.118083,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,...,1.858865e-03,1.858864e-03,1.858864e-03,1.858865e-03,1.858862e-03,1.992449e-05,1.868443e-03,0.003531,3.531138e-03,1.858833e-03
202763,194334621,49,2,-0.050308,-0.112834,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,...,1.707288e-03,1.707287e-03,1.707287e-03,1.707288e-03,1.707285e-03,4.973969e-06,1.718449e-03,0.003243,3.243191e-03,1.707255e-03
202764,194335206,49,2,-0.154605,-0.116753,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,...,2.557984e-04,2.557972e-04,2.557972e-04,2.557984e-04,2.557948e-04,1.933202e-04,2.232165e-04,0.000486,4.858661e-04,2.557661e-04
202765,194335216,49,2,0.708000,-0.118083,1.018602,-0.121970,-0.043875,-0.113002,-0.061584,...,1.226060e-02,1.226060e-02,1.226060e-02,1.226060e-02,1.226060e-02,5.065905e-05,1.233830e-02,0.023291,2.329083e-02,1.226057e-02


In [9]:
# Goal: binary classification of 0,1
# 0: licit, 1: illicit

X_data = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] < 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_training_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data)]
X_train = X_training_timesteps.drop(columns=['txId', 'class', 'Time step'])

X_data_test = df_txs_features_selected.loc[(df_txs_features_selected['Time step'] >= 35) & (df_txs_features_selected['class'] != 3), 'txId']
X_testing_timesteps = df_txs_features_selected.loc[df_txs_features_selected['txId'].isin(X_data_test)]
X_test = X_testing_timesteps.drop(columns=['txId', 'class', 'Time step'])

y_training_timesteps = X_training_timesteps[['class']]
y_training_timesteps = y_training_timesteps['class'].apply(lambda x: 0 if x == 2 else 1 ) # change illicit (class-2) to '0' for classification
y_train = y_training_timesteps

y_testing_timesteps = X_testing_timesteps[['class']]
y_testing_timesteps = y_testing_timesteps['class'].apply(lambda x: 0 if x == 2 else 1 ) # change illicit (class-2) to '0' for classification
y_test = y_testing_timesteps

In [10]:
# LOGISTIC REGRESSION (LR)
cLR = LogisticRegression(max_iter=1000).fit(X_train.values,y_train.values)
y_preds_LR = cLR.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_LR)

print("Logistic Regression")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_LR, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))

Logistic Regression
Precision: 0.323 
Recall: 0.704 
F1 Score: 0.443
Micro-Average F1 Score: 0.883


In [11]:
# RANDOM FOREST (RF)
cRF = RandomForestClassifier(n_estimators=50).fit(X_train.values,y_train.values)
y_preds_RF = cRF.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_RF)

print("Random Forest")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_RF, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))

Random Forest
Precision: 0.965 
Recall: 0.720 
F1 Score: 0.825
Micro-Average F1 Score: 0.980


In [12]:
# MULTILAYER PERCEPTRON (MLP)
cMLP = MLPClassifier(solver='adam', learning_rate_init=0.001, max_iter=200).fit(X_train.values,y_train.values)
y_preds_MLP = cMLP.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_MLP)

print("Multilayer Perceptron (MLP)")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_MLP, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))

Multilayer Perceptron (MLP)
Precision: 0.593 
Recall: 0.690 
F1 Score: 0.638
Micro-Average F1 Score: 0.948


In [13]:
# XGBOOST (XGB)
cXGB = xgb.XGBClassifier(objective="multi:softmax", num_class=2, random_state=42)
cXGB.fit(X_train.values, y_train.values)
y_preds_XGB = cXGB.predict(X_test.values)
prec,rec,f1,num = precision_recall_fscore_support(y_test.values, y_preds_XGB)

print("XGBOOST")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f"%(prec[1],rec[1],f1[1]))
micro_f1 = f1_score(y_test, y_preds_XGB, average='micro')
print("Micro-Average F1 Score: %.3f"%(micro_f1))
#print(confusion_matrix(y, y_pred))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_class=2, ...)

XGBOOST
Precision: 0.922 
Recall: 0.730 
F1 Score: 0.815
Micro-Average F1 Score: 0.978


In [14]:
# LIGHTGBM (LGBM)
import lightgbm as lgb
from sklearn.metrics import precision_recall_fscore_support, f1_score

# Initialize classifier with class_weight BEFORE fitting
cLGBM = lgb.LGBMClassifier(n_estimators=50, class_weight='balanced')
cLGBM.fit(X_train.values, y_train.values)

# Predict
y_preds_LGBM = cLGBM.predict(X_test.values)

# Compute metrics
prec, rec, f1, num = precision_recall_fscore_support(y_test.values, y_preds_LGBM)

print("LightGBM")
print("Precision: %.3f \nRecall: %.3f \nF1 Score: %.3f" % (prec[1], rec[1], f1[1]))

micro_f1 = f1_score(y_test, y_preds_LGBM, average='micro')
print("Micro-Average F1 Score: %.3f" % (micro_f1))


C:\Users\Connect2Aryans\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning:

Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.

  File "C:\Users\Connect2Aryans\AppData\Local\Programs\Python\Python313\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "C:\Users\Connect2Aryans\AppData\Local\Programs\Python\Python313\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Connect2Aryans\AppData\Local\Programs\Python\Python313\Lib\subp

[LightGBM] [Info] Number of positive: 3462, number of negative: 26237
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015879 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 41951
[LightGBM] [Info] Number of data points in the train set: 29699, number of used features: 181
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


LGBMClassifier(class_weight='balanced', n_estimators=50)

LightGBM
Precision: 0.608 
Recall: 0.740 
F1 Score: 0.667
Micro-Average F1 Score: 0.951


C:\Users\Connect2Aryans\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but LGBMClassifier was fitted with feature names

